## Miwok Entrants Database

### This notebook reads in the entrants table from miwok100k website.

### To Do : Scrape ultrasignup for the people listed here & store info in a Pandas database

In [1]:
from bs4 import BeautifulSoup
import urllib2
import codecs

response = urllib2.urlopen('http://miwok100k.com/site/lottery/')
html = response.read()
soup = BeautifulSoup(html)

entrant_table = soup.find("table", {"class" : "tableizer-table"})

/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /Users/stephanie/anaconda3/envs/snakes/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
foo = entrant_table.findAll('tr')
header = foo[0]
header

<tr class="tableizer-firstrow">\n<th>First Name</th>\n<th>Last Name</th>\n<th>City</th>\n<th>State</th>\n</tr>

In [3]:
row_one = foo[1]
row_one

<tr>\n<td>Jeffrey</td>\n<td>Abando</td>\n<td>Baltimore</td>\n<td>MD</td>\n</tr>

In [4]:
entrants = foo[1:]

In [5]:
records = [] # store all of the records in this list
for row in entrants:
    col = row.findAll('td')
    firstname = col[0].string.strip()
    lastname = col[1].string.strip()
    city = col[2].string.strip()
    state = col[3].string.strip()
    record = '%s,%s,%s,%s' % (firstname, lastname, city, state) # store the record, comma separated
    records.append(record)

# If you want to save a text file:
#fl = codecs.open('output.txt', 'wb', 'utf8')
#line = ';'.join(records)
#fl.write(line + u'\r\n')
#fl.close()

In [6]:
records[:20]

[u'Jeffrey,Abando,Baltimore,MD',
 u'Lynn,Adami,San Mateo,CA',
 u'Eleanore,Adams,albany,CA',
 u'Nick,Affleck,Chico,CA',
 u'Eric,Aldrich,Santa Cruz,CA',
 u'Jim,Alexander,Roseville,CA',
 u'Chuck,Amital,San Francisco,CA',
 u'Bobb,Ankeney,placerville,CA',
 u'Megan,Arauzo,Novato,CA',
 u'Jefferson,Arbuckle,Boulder,CO',
 u'John,Arledge,los altos,CA',
 u'Jeanie,Arnold,Seattle,WA',
 u'Michael,Arnstein,Kailua,HI',
 u'Michael,Arsenault,Middleton,NH',
 u'Moises,Avila,Gilroy,CA',
 u'Michael,Baikie,Mountain View,CA',
 u'Trevor,Baine,Calgary,AB',
 u'Emile,Baizel,San Francisco,CA',
 u'Kathleen,Baker,Kailua,HI',
 u'Audrey,Baldessari,Redmond,WA']

In [7]:
import json
response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/Stephanie/Wuerth/')
data = json.loads(response.read())
print "Rank: ", data[0]['Rank']
print "AgeRank: ", data[0]['AgeRank']

#data[0]
data[0]

Rank:  0.8614
AgeRank:  0.9291


{u'Age': 27,
 u'AgeRank': 0.9291,
 u'FirstName': u'Stephanie',
 u'Gender': u'F',
 u'Id': 133072,
 u'ImageId': u'f50cabbe-d91e-4429-bc75-598c90ec807a',
 u'LastName': u'Wuerth',
 u'Rank': 0.8614,
 u'Results': [{u'age': 27,
   u'age_rank': 0,
   u'city': u'Oakland',
   u'drilldown': 1,
   u'event_distance_id': 37909,
   u'eventdate': u'10/29/2016 12:00:00 AM',
   u'eventname': u'Oakland Hills Trail Run - 35K',
   u'formattime': u'4:06:56',
   u'gender_place': 5,
   u'photo_count': 0,
   u'place': 12,
   u'prior_count': 0,
   u'race_count': 0,
   u'runner_rank': 0.7813,
   u'state': u'CA',
   u'status': 1,
   u'time': u'4:06:56'},
  {u'age': 27,
   u'age_rank': 0,
   u'city': u'Ashland',
   u'drilldown': 1,
   u'event_distance_id': 35902,
   u'eventdate': u'7/23/2016 12:00:00 AM',
   u'eventname': u'Siskiyou Out Back - 50 Miler',
   u'formattime': u'9:08:50',
   u'gender_place': 1,
   u'photo_count': 0,
   u'place': 19,
   u'prior_count': 0,
   u'race_count': 0,
   u'runner_rank': 1.0,
   

In [8]:
response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/jake/edman/') #lowercase works!
data = json.loads(response.read())
print "Rank: ", data[0]['Rank']
print "AgeRank: ", data[0]['AgeRank']

#data[0]

Rank:  0.8155
AgeRank:  0.8531


In [9]:
response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/Serena/Eley/')
data = json.loads(response.read())
print "Rank: ", data[0]['Rank']
print "AgeRank: ", data[0]['AgeRank']

#data[0]

Rank:  0.9703
AgeRank:  0.9746
